In [ ]:

import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nlp_id.lemmatizer import Lemmatizer
from wordcloud import WordCloud
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [5]:
df = pd.read_csv("../data/hasil_scraping_demo/fix_scarap/cleaned_data.csv")

In [11]:
df

,created_at,favorite_count,full_text,quote_count,reply_count,retweet_count,hashtags,cleaned_text
0,Sun Aug 31 14:43:43 +0000 2025,5677,Menyikapi pernyataan Prabowo bersama dengan pi...,54,87,3660,[],menyikapi pernyataan prabowo bersama dengan pi...
1,Sun Aug 31 10:11:46 +0000 2025,7435,Saudara-saudari sebangsa dan setanah air Negar...,4037,4287,1575,[],saudarasaudari sebangsa dan setanah air negara...
2,Fri Aug 29 06:27:34 +0000 2025,14161,Ringkasan Pernyataan Presiden: 1. Menuduh demo...,2003,709,14116,[],ringkasan pernyataan presiden menuduh demonst...
3,Sat Aug 30 10:34:24 +0000 2025,4806,Musuh kita bukan fasilitas umum. Bukan perbeda...,34,65,1939,[],musuh kita bukan fasilitas umum bukan perbedaa...
4,Sat Aug 30 06:44:02 +0000 2025,306,R E Q U I E M ● Brutalitas aparat bukan saja p...,1,30,93,[],requiem brutalitas aparat bukan saja problem ...
...,...,...,...,...,...,...,...,...
1218,Thu Aug 28 07:51:29 +0000 2025,2541,Hari ini mahasiswa melakukan aksi demo lagi di...,17,41,562,[],hari ini mahasiswa melakukan aksi demo lagi di...
1219,Fri Aug 29 11:33:23 +0000 2025,1557,Demo Bubarkan DPR Evaluasi Pejabat Korup dan T...,15,23,540,[],demo bubarkan dpr evaluasi pejabat korup dan t...
1220,Mon Aug 25 05:43:50 +0000 2025,4135,Pantauan Demo di Gedung DPR RI tanggal 25 Agus...,48,49,1120,[],pantauan demo di gedung dpr ri tanggal agustu...
1221,Sun Aug 31 22:36:49 +0000 2025,4441,SEPULUH TUNTUTAN RAKYAT UNTUK INDONESIA (SEPUL...,65,299,2135,[],sepuluh tuntutan rakyat untuk indonesia sepult...


In [ ]:
from typing import List
import matplotlib.pyplot as plt

In [ ]:
def plot_wordcloud(texts:list):
    text = " ".join(texts)
    wordcloud = WordCloud(width=800, height=400, background_color="white").generate(text)

    plt.figure(figsize=(10,5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title("WordCloud", fontsize=16)
    plt.show()

def plot_top_words(texts:list, n:int=10):
    all_words = " ".join(texts).split()
    counter = Counter(all_words)
    most_common = counter.most_common(n)

    words = [w for w, _ in most_common]
    freqs = [f for _, f in most_common]

    plt.figure(figsize=(8,5))
    plt.barh(words[::-1], freqs[::-1])
    plt.xlabel("Frekuensi")
    plt.title(f"Top {n} Kata Paling Sering Muncul")
    plt.show()


In [ ]:
def dict_database(df, kolom_input, kolom_output, path_kamus_alay, path_stopwords, path_abusive):
    try:
        kamus_alay_df = pd.read_csv(path_kamus_alay, header=None, names=['alay', 'baku'], encoding='latin1')
        kamus_normalisasi = dict(zip(kamus_alay_df['alay'], kamus_alay_df['baku']))

        with open(path_stopwords, 'r') as file:
            stop_words = set(line.strip() for line in file)

        with open(path_abusive, 'r') as file:
            abusive_words = set(line.strip() for line in file)
            
        print("File kamus, stopwords, dan abusive berhasil dimuat.")
    except FileNotFoundError as e:
        print(f"Error: File resource tidak ditemukan - {e.filename}. Proses dibatalkan.")
        return df

    def proses_pembersihan_teks(text):
        text = str(text).lower()
        text = re.sub(r'https?://\S+|www\.\S+', '', text)
        text = re.sub(r'@[A-Za-z0-9_]+|#[A-Za-z0-9_]+', '', text)
        text = re.sub(r'[^a-z\s]', '', text)
        
        tokens = text.split()
        
        cleaned_tokens = []
        for token in tokens:
            token = kamus_normalisasi.get(token, token)
            if token and token not in abusive_words and token not in stop_words:
                cleaned_tokens.append(token)
                
        return ' '.join(cleaned_tokens)

    print(f"Memproses kolom '{kolom_input}'...")
    df[kolom_output] = df[kolom_input].apply(proses_pembersihan_teks)
    print("Proses pembersihan selesai.")
    
    return df

try:
    df_utama = pd.read_csv("../data/hasil_scraping_demo/fix_scarap/cleaned_data.csv")

    path_kamus = '../data/kamusalay.csv'
    path_stopword = '../data/stopwords-id.txt'
    path_abusive_word = '../data/abusive-id.txt'

    df_hasil = dict_database(
        df=df_utama,
        kolom_input='cleaned_text',
        kolom_output='cleaned_text_2',
        path_kamus_alay=path_kamus,
        path_stopwords=path_stopword,
        path_abusive=path_abusive_word
    )

    print("\nBerikut adalah perbandingannya:")
    print(df_hasil[['cleaned_text', 'cleaned_text_2']].head())

except FileNotFoundError:
    print("Error: File 'cleaned_data.csv' tidak ditemukan. Harap periksa path file Anda.")

Semua file berhasil dimuat.

Proses pembersihan selesai. Berikut adalah perbandingannya:
                                           full_text  \
0  Menyikapi pernyataan Prabowo bersama dengan pi...   
1  Saudara-saudari sebangsa dan setanah air Negar...   
2  Ringkasan Pernyataan Presiden: 1. Menuduh demo...   
3  Musuh kita bukan fasilitas umum. Bukan perbeda...   
4  R E Q U I E M ● Brutalitas aparat bukan saja p...   

                                        cleaned_text  
0  menyikapi pernyataan prabowo bersama dengan pi...  
1  saudarasaudari sebangsa dan setanah air negara...  
2  ringkasan pernyataan presiden  menuduh demonst...  
3  musuh kita bukan fasilitas umum bukan perbedaa...  
4  requiem  brutalitas aparat bukan saja problem ...  


In [14]:
df

,created_at,favorite_count,full_text,quote_count,reply_count,retweet_count,hashtags,cleaned_text,cleaned_text_2
0,Sun Aug 31 14:43:43 +0000 2025,5677,Menyikapi pernyataan Prabowo bersama dengan pi...,54,87,3660,[],menyikapi pernyataan prabowo bersama dengan pi...,menyikapi pernyataan prabowo pimpinan lembaga ...
1,Sun Aug 31 10:11:46 +0000 2025,7435,Saudara-saudari sebangsa dan setanah air Negar...,4037,4287,1575,[],saudarasaudari sebangsa dan setanah air negara...,saudarasaudari sebangsa setanah air negara men...
2,Fri Aug 29 06:27:34 +0000 2025,14161,Ringkasan Pernyataan Presiden: 1. Menuduh demo...,2003,709,14116,[],ringkasan pernyataan presiden menuduh demonst...,ringkasan pernyataan presiden menuduh demonstr...
3,Sat Aug 30 10:34:24 +0000 2025,4806,Musuh kita bukan fasilitas umum. Bukan perbeda...,34,65,1939,[],musuh kita bukan fasilitas umum bukan perbedaa...,musuh fasilitas perbedaan suku ras agama musuh...
4,Sat Aug 30 06:44:02 +0000 2025,306,R E Q U I E M ● Brutalitas aparat bukan saja p...,1,30,93,[],requiem brutalitas aparat bukan saja problem ...,requiem brutalitas aparat problem struktural k...
...,...,...,...,...,...,...,...,...,...
1218,Thu Aug 28 07:51:29 +0000 2025,2541,Hari ini mahasiswa melakukan aksi demo lagi di...,17,41,562,[],hari ini mahasiswa melakukan aksi demo lagi di...,mahasiswa aksi demo didepan gedung dpr ri aksi...
1219,Fri Aug 29 11:33:23 +0000 2025,1557,Demo Bubarkan DPR Evaluasi Pejabat Korup dan T...,15,23,540,[],demo bubarkan dpr evaluasi pejabat korup dan t...,demo bubarkan dpr evaluasi pejabat korup tinda...
1220,Mon Aug 25 05:43:50 +0000 2025,4135,Pantauan Demo di Gedung DPR RI tanggal 25 Agus...,48,49,1120,[],pantauan demo di gedung dpr ri tanggal agustu...,pantauan demo gedung dpr ri tanggal agustus ti...
1221,Sun Aug 31 22:36:49 +0000 2025,4441,SEPULUH TUNTUTAN RAKYAT UNTUK INDONESIA (SEPUL...,65,299,2135,[],sepuluh tuntutan rakyat untuk indonesia sepult...,sepuluh tuntutan rakyat indonesia sepultura de...
